# PID Classification with Neural Networks (Unsupervised Training)

This example illustrates the classification of particle types using [tensorflow](https://www.tensorflow.org/)/[neupy](http://neupy.com/pages/home.html) neural networks. The unsupervised training uses a Growing Neural Gas (GNG) model with MC generated data of [BaBar](https://www.flickr.com/photos/slaclab/46211844232). As the training might take a very long time, it is possible to persist a network and read the trained network for data analytics.

We begin with the standard imports. This notebook uses numpy, pandas, seaborn, matplotlib, and tensorflow/keras. The data are read from a ROOT file with uproot:

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import uproot

print(tf.__version__)

# Training Dataset
We are reading the training dataset from a ROOT file. The file contains particle momentum (mom) and track elevation (theta), dE/dx measurement from silicon vertex tracker (svt) and drift chamber (dch), as well as energy deposit in the electromagnetic calorimeter (emc), the cerenkov angle in the DIRC (drc), and the hit patterns in the instrumented flux return (ifr). The file in addition holds higher level features like partial energy sums, zernicke momenta, likelihood etc. 

The particles are labeled (id)
* Electron = 0
* Muon = 1
* Pion = 2
* Kaon = 3
* Proton = 4

In [ ]:
target_names = ['Electron', 'Muon', 'Pion', 'Kaon', 'Proton']

In [ ]:
file = uproot.open("pid.root")

In [ ]:
file.keys()

In [ ]:
tree = file["PidTuple"]
tree.keys()

In [ ]:
tree.numentries

We assemble the training dataset as a pandas dataframe and plot the particle statistics:

In [ ]:
data = tree.arrays(["id", "mom", "theta", "svt", "emc", "drc", "dch", "ifr"])
dataset = pd.DataFrame(data)
dataset.tail()

In [ ]:
electron = dataset[dataset[b'id']==0][b'id'].value_counts()
muon = dataset[dataset[b'id']==1][b'id'].value_counts()
pion = dataset[dataset[b'id']==2][b'id'].value_counts()
kaon = dataset[dataset[b'id']==3][b'id'].value_counts()
proton = dataset[dataset[b'id']==4][b'id'].value_counts()

df = pd.DataFrame([electron, muon, pion, kaon, proton])
df.index = target_names
df.plot(kind='bar',stacked=True, figsize=(10,5))

# Data Preparation
We generate a training dataset (90%) and a test dataset (10%) from the input dataset. The vectors are shuffled in random order.

In [ ]:
train_dataset = dataset.sample(frac=0.9,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

Show a correlation matrix of the feature vector:

In [ ]:
train = train_dataset[[b'mom', b'theta', b'svt', b'emc', b'drc', b'dch', b'ifr', b'id']]
train[b'id'] = train[b'id'].map( {0: 'electron', 1: 'muon', 2: 'pion', 3:'kaon', 4:'proton'} ).astype(str)
sns.pairplot(data=train[:1000], hue=b'id', diag_kind="kde")
plt.show()

We extract the particle labels to be used for particle tagging:

In [ ]:
train_labels = train_dataset.pop(b'id')
test_labels = test_dataset.pop(b'id')

We normalize the training and test vectors:

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

## 1. Build and train the GNG Model
The keras model implements a neural network with two hidden layers. The input feature vector represents measured detector quantities as defined above. The output reflects the particle probabilities.

In [ ]:
from neupy import algorithms, utils
def build_model(dimension, max_nodes=10000, step=0.5, n_start_nodes=2, max_edge_age=15):
    model = algorithms.GrowingNeuralGas(
        n_inputs=dimension,
        n_start_nodes=n_start_nodes,

        shuffle_data=True,
        verbose=True,

        step=step,
        neighbour_step=0.05,

        max_edge_age=max_edge_age,
        max_nodes=max_nodes,

        n_iter_before_neuron_added=10,
        after_split_error_decay_rate=0.1,
        error_decay_rate=0.995,
        min_distance_for_update=0.01,
    )
    return model

In [ ]:
model = build_model(len(train_dataset.keys()))

Train the GNG network (This might take a _very_ long time, depending on your CPU power). Draw the GNG network for each iteration.

In [ ]:
utils.reproducible()
for epoch in range(10):
    model.train(train_dataset, epochs=1)

Save the GNG network

In [ ]:
import dill
with open('gng.dill', 'wb') as f:
    dill.dump(model, f)

## 2. Analyze the GNG Network
We identify the clusters in the network that represent the particle types. 

In [ ]:
import dill
with open('gng.dill', 'rb') as f:
    model = dill.load(f)

In [ ]:
len(model.graph.nodes)

In [ ]:
len(model.graph.edges)

Run a k-means algorithm to identify clusters of nodes. We use 5 categories as we have 5 particle types:

In [ ]:
from sklearn.cluster import KMeans
train = train_dataset.to_numpy()
nodes = model.graph.nodes
weights = np.concatenate([node.weight for node in nodes])
kmeans = KMeans(n_clusters=5, random_state=0).fit(weights)
kpred_labels = kmeans.predict(train)
kpredictions = tf.keras.utils.to_categorical(weights)

In [ ]:
def winner_node(graph,sample,n=1):
    nodes = graph.nodes
    weights = np.concatenate([node.weight for node in nodes])
    distance = np.linalg.norm(weights - sample, axis=1)
    neuron_ids = np.argsort(distance)
    closest_neuron_id = neuron_ids[0:n]
    return closest_neuron_id

In [ ]:
def node_distance(graph,sample):
    nodes = graph.nodes
    weights = np.concatenate([node.weight for node in nodes])
    distance = np.linalg.norm(weights - sample, axis=1)
    neuron_ids = np.argsort(distance)
    closest_neuron_id, second_closest_id = neuron_ids[:2]
    closest_neuron = nodes[closest_neuron_id]
    second_closest = nodes[second_closest_id]
    total_error = 0
    for to_neuron in list(graph.edges_per_node[closest_neuron]): 
        edge_id = graph.find_edge_id(to_neuron, closest_neuron)
        #print(edge_id)
        total_error += distance[graph.edges[edge_id]] 
    return closest_neuron_id,total_error

In order to determine the responsibility of the graph nodes wrt. a certain hypothesis we assign the training labels to the corresponding winner nodes of the training data. This is achieved by fillling a 2D histogram with the winner nodes vs. the training labels.

In [ ]:
bestmatches = 2
labels  = np.repeat(train_labels,bestmatches)
winners = np.arange(0)
for sample in train:
    winners = np.append(winners, winner_node(model.graph,sample,bestmatches))

It seems that winner nodes are peaking at different locations for the various particle types.

In [ ]:
#labels = np.repeat(train_labels,bestmatches)
histogram, xedges, yedges = np.histogram2d(winners,labels,bins=(len(model.graph.nodes),5))
plt.hist2d(winners,labels,bins=(len(model.graph.nodes),5),cmap='Greys')
plt.show()

In order to make a prediction for a sample feature vector of the test dataset we determine the histogram counts for each particle hypothesis and transform the five values to a softmax probability vector.

In [ ]:
def predict(graph,sample,histogram):
    w = winner_node(graph,sample)[0]  # winner_node returns a list of the first n winners, default is n=1
    pred = np.array(histogram[w])  # [el, mu, pi, ka, pr]
    pred /= (np.amax(pred) + 1.e-6)
    pexp = np.exp(pred)
    return pexp / np.sum(pexp)  # softmax

In [ ]:
test = test_dataset.to_numpy()
pred_labels = np.arange(0)
predictions = np.arange(0)
for sample in test:
    prediction = predict(model.graph,sample,histogram)
    predictions = np.append(predictions, prediction)
    label = np.where(prediction == np.amax(prediction))
    pred_labels = np.append(pred_labels, label[0][0]) # Take only the first element in case of ambiguities

predictions = np.reshape(predictions, (-1, 5))
#predictions = to_categorical(pred_labels)

In [ ]:
predictions

## 3. Model Evaluation
We evaluate the model performance using the mormalized test data set and compare the output vector to the test labels. The output vector holds the probabilities of the five particle hypotheses.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
#pred_labels = np.argmax(predictions, axis=1)
print(confusion_matrix(test_labels, pred_labels))

In [ ]:
print(classification_report(test_labels, pred_labels, target_names=target_names))

## 4. Physics Control Plots

We inspect the results by plotting the measured data in dependance of the particle momentum. The color index is: protons (yellow), kaons (green), pions (cyan), muons (blue), electrons (red).

In [ ]:
mom = test_dataset[b'mom']
svt = test_dataset[b'svt']
emc = test_dataset[b'emc']
dch = test_dataset[b'dch']
drc = test_dataset[b'drc']
ifr = test_dataset[b'ifr']

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(mom, svt, alpha=0.5,
            s=5, c=test_labels, cmap='viridis')
plt.ylabel('dE/dx SVT')
plt.xlabel('momentum [GeV/c]')
plt.subplot(1,2,2)
plt.scatter(mom, svt, alpha=0.5,
            s=5, c=pred_labels, cmap='viridis')
plt.ylabel('dE/dx SVT')
plt.xlabel('momentum [GeV/c]')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(mom, emc, alpha=0.5,
            s=5, c=test_labels, cmap='viridis')
plt.ylabel('Energy [GeV]')
plt.xlabel('momentum [GeV/c]')
plt.subplot(1,2,2)
plt.scatter(mom, emc, alpha=0.5,
            s=5, c=pred_labels, cmap='viridis')
plt.ylabel('Energy [GeV]')
plt.xlabel('momentum [GeV/c]')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(mom, dch, alpha=0.5,
            s=5, c=test_labels, cmap='viridis')
plt.ylabel('dch')
plt.xlabel('momentum [GeV/c]')
plt.subplot(1,2,2)
plt.scatter(mom, dch, alpha=0.5,
            s=5, c=pred_labels, cmap='viridis')
plt.ylabel('dch')
plt.xlabel('momentum [GeV/c]')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(mom, drc, alpha=0.5,
            s=5, c=test_labels, cmap='viridis')
plt.ylabel('drc')
plt.xlabel('momentum [GeV/c]')
plt.subplot(1,2,2)
plt.scatter(mom, drc, alpha=0.5,
            s=5, c=pred_labels, cmap='viridis')
plt.ylabel('drc')
plt.xlabel('momentum [GeV/c]')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(mom, ifr, alpha=0.5,
            s=10, c=test_labels, cmap='viridis')
plt.ylabel('Layers')
plt.xlabel('momentum [GeV/c]')
plt.subplot(1,2,2)
plt.scatter(mom, ifr, alpha=0.5,
            s=10, c=pred_labels, cmap='viridis')
plt.ylabel('Layers')
plt.xlabel('momentum [GeV/c]')
plt.show()

## Muon Selection
By inspection of the physics control plots it seems difficult to clearly separate muons (blue) from pions (cyan) by simple linear cuts. Thus we want to construct a muon selector based on the network output respecting the probability of the five particle hypotheses. Taking into account the relative a priori probabilities of the particle occurence we can formulate a likelihood ratio to observe a muon (Pions are 5 times more abundant than muons, protons occur at 10% only).

In [ ]:
electron = predictions[:,0]
muon     = predictions[:,1]
pion     = predictions[:,2] * 5.0
kaon     = predictions[:,3]
proton   = predictions[:,4] * 0.1

In [ ]:
L = np.log(muon) - np.log(pion)

If we plot L = log(muon) - log(pion) we are able to clearly separate muons by a cut on L > -2.0 over a wide momentum range.

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(mom, L, alpha=0.2,
            s=10, c=test_labels, cmap='viridis')
plt.ylabel('log(muon) - log(pion)')
plt.xlabel('momentum [GeV]')
plt.subplot(1,2,2)
plt.scatter(mom, L, alpha=0.2,
            s=10, c=pred_labels, cmap='viridis')
plt.ylabel('log(muon) - log(pion)')
plt.xlabel('momentum [GeV]')
plt.show()